# 🎬 AI Storyboard Weaver for Filmmakers 2.0

**Capstone Project: Gen AI Intensive Course 2025Q1**  
*Author: Aya Nabil*  
*Date: April 09, 2025*  
*Version: 2.0 (Enhanced with Agents & RAG)*

## 🎯 Enhanced Capstone Goal & GenAI Capabilities

This enhanced version implements a professional Automated Storyboard Weaver with:

✅ **Agent-Based Function Calling System**  
- Proper orchestration of generation tasks  
- Specialized functions for each operation  
- Error handling and validation  

✅ **True RAG Pipeline**  
- Sentence Transformers for embeddings  
- Cosine similarity for plot retrieval  
- Persistent knowledge base  

✅ **Improved UI/UX**  
- Progress indicators  
- Enhanced visualizations  
- Better error handling 

## 🛠️ Technical Architecture
    
```mermaid
graph TD
    A[User Input] --> B[StoryboardAgent]
    B --> C[Generate Storyboard]
    B --> D[Analyze Mood]
    B --> E[Retrieve Similar Plots]
    C --> F[Gemini API]
    E --> G[Knowledge Base]
    G --> H[Embeddings Store]
    F --> I[Visualization]
    D --> I
```

In [1]:
# Standard library imports for basic functionality
import json  # For handling JSON data (e.g., API responses, knowledge base)
import re  # For regular expressions to parse API output
import warnings  # To suppress unnecessary warnings for cleaner output
from typing import Dict, List, Optional, Tuple  # For type hints to improve code readability
import os  # For file system operations (e.g., checking if knowledge base exists)
from pathlib import Path  # For handling file paths in a cross-platform way
from datetime import datetime  # For timestamping knowledge base entries

# Third-party imports for external functionality
import requests  # For making HTTP requests to the Gemini API and Wikipedia
from bs4 import BeautifulSoup  # For parsing HTML content from Wikipedia
import matplotlib.pyplot as plt  # For creating visualizations (e.g., mood distribution chart)
import numpy as np  # For numerical operations (e.g., array handling in RAG)
from IPython.display import display, Markdown, HTML  # For rendering markdown and HTML in Jupyter
from ipywidgets import widgets, Layout  # For creating interactive UI elements (e.g., text input, buttons)
import matplotlib.colors as mcolors  # For accessing predefined color sets for visualizations
from matplotlib import cm  # For color mapping in visualizations

# RAG components for retrieval-augmented generation
from sentence_transformers import SentenceTransformer  # For generating embeddings of text (plots)
from sklearn.metrics.pairwise import cosine_similarity  # For calculating similarity between embeddings

# Suppress warnings to keep the notebook output clean
warnings.filterwarnings('ignore')

# Configuration dictionary with enhanced settings for the application
CONFIG = {
    "api_url": "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent",  # URL for Gemini API with updated model
    "max_scenes": 5,  # Maximum number of scenes allowed in a storyboard
    "moods": ["tense", "joyful", "romantic", "suspenseful", "chaotic", "dark", "hopeful"],  # List of possible moods for scenes
    "context_length": 300,  # Maximum length of context text fetched from Wikipedia
    "colors": list(mcolors.TABLEAU_COLORS.values()),  # Colors for visualizations (e.g., mood chart)
    "max_retries": 3,  # Maximum number of retries for API calls
    "rag_threshold": 0.7,  # Similarity threshold for RAG retrieval (cosine similarity)
    "knowledge_base": "knowledge_base.json",  # File path for persistent storage of RAG data
    "embedding_model": "all-MiniLM-L6-v2",  # Sentence embedding model for RAG
    "default_genre": "drama"  # Fallback genre if detection fails
}

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

## 🔐 Secure API Configuration
**Security Note:** API keys should never be hardcoded or exposed in notebooks.  
This implementation uses:
- IPython's Password widget to prevent key visibility
- Environment variables for deployment
- Secure handling throughout the pipeline

In [ ]:
# Create a password widget for secure API key input to avoid hardcoding sensitive information
api_key = widgets.Password(
    placeholder='Enter your Gemini API key',  # Placeholder text for the input field
    description='API Key:',  # Label for the input field
    layout=Layout(width='400px')  # Set the width of the widget for better UI
)

# Display the API key input widget in the notebook
display(api_key)

## 🤖 Storyboard Agent Architecture
The agent system provides:
- Orchestration of generation tasks
- Specialized function calling
- Error handling pipeline
- Quality validation

In [ ]:
class StoryboardAgent:
    """Main agent class that manages the storyboard generation pipeline with RAG and API integration"""

    def __init__(self):
        # Initialize the embedding model for RAG to convert plots into vector embeddings
        try:
            self.embedding_model = SentenceTransformer(CONFIG["embedding_model"])  # Load the sentence transformer model
        except Exception as e:
            print(f"⚠️ Could not load embedding model: {e}")  # Print error if model loading fails
            self.embedding_model = None  # Set to None to disable RAG if model fails
        
        # Initialize the knowledge base for storing plots and storyboards
        self._init_knowledge_base()
        
        # Register available functions for the agent to call dynamically
        self.available_functions = {
            "generate_storyboard": self.generate_storyboard,  # Function to generate a storyboard
            "analyze_mood": self.analyze_mood,  # Function to analyze mood distribution
            "retrieve_similar_plots": self.retrieve_similar_plots,  # Function to retrieve similar plots using RAG
            "save_storyboard": self.save_storyboard,  # Function to save the storyboard to a file
            "validate_storyboard": self.validate_storyboard  # Function to validate the storyboard structure
        }

    def _init_knowledge_base(self):
        """Initialize or load the RAG knowledge base from a JSON file"""
        # Check if the knowledge base file exists
        if not os.path.exists(CONFIG["knowledge_base"]):
            # If it doesn't exist, create a new JSON file with empty lists for scripts and plots
            with open(CONFIG["knowledge_base"], "w") as f:
                json.dump({"scripts": [], "plots": []}, f)  # Initialize with empty structure

    def execute_function(self, function_name: str, **kwargs):
        """Execute a registered function with error handling and progress feedback"""
        # Check if the requested function exists in the registry
        if function_name not in self.available_functions:
            raise ValueError(f"Unknown function: {function_name}")  # Raise an error if function is not found
        
        try:
            # Display a progress message to the user
            display(Markdown(f"🔧 Executing {function_name.replace('_', ' ')}..."))
            # Call the requested function with the provided arguments
            return self.available_functions[function_name](**kwargs)
        except Exception as e:
            # Display an error message if the function fails
            display(Markdown(f"⚠️ **Error in {function_name}:** {str(e)}"))
            return None  # Return None to indicate failure

    def generate_storyboard(self, plot: str, num_scenes: int = 3) -> Dict:
        """Generate a storyboard using the Gemini API with RAG context for enhanced generation"""
        # Retrieve similar plots from the knowledge base to provide context
        similar_plots = self.retrieve_similar_plots(plot)
        # Format the similar plots into a string for inclusion in the prompt
        rag_context = "\nSimilar plots:\n" + "\n".join(
            [f"- {p['plot']}" for p in similar_plots[:2]]) if similar_plots else ""
        
        # Build the prompt for the Gemini API with the plot and RAG context
        prompt = self._build_prompt(plot, num_scenes, rag_context)
        # Call the Gemini API to generate the storyboard
        storyboard = self._call_generation_api(prompt)
        
        # If the storyboard was successfully generated, update the knowledge base
        if storyboard:
            self._update_knowledge_base(plot, storyboard)
        return storyboard

    def _build_prompt(self, plot: str, num_scenes: int, rag_context: str = "") -> str:
    """Construct the generation prompt with dynamic context for the Gemini API"""
    # Detect the genre of the plot to fetch relevant context
    genre = self.detect_genre(plot)
    # Fetch genre-specific context from Wikipedia
    wiki_context = self.fetch_wikipedia_film_data(genre)
    # Fetch a sample script dialogue for the genre from the mock database
    script_example = self.fetch_script_data(genre)
    
    # Get the visual style from the style_selector widget
    visual_style = style_selector.value.lower()
    
    # Add instructions for documentary style if selected
    style_instruction = ""
    if visual_style == "documentary":
        style_instruction = """
        - Use a documentary visual style: descriptions should feel realistic and observational, as if filmed by a camera crew documenting real events.
        - Include elements like grainy footage, voiceover narration, location captions, or references to archival footage.
        - Dialogue should be naturalistic, like interviews or recorded conversations, with historical context relevant to 1920s Paris (e.g., post-World War I, the art scene in Montmartre, the rise of jazz).
        """
    
    # Construct and return the prompt with all required components
    return f"""Generate a film storyboard in JSON format based on: "{plot}"
    Format Requirements:
    - Strictly valid JSON output
    - Title reflecting plot essence
    - {num_scenes} scenes with:
      • scene_number (1-{num_scenes})
      • vivid description
      • natural dialogue
      • mood from: {CONFIG['moods']}
    Visual Style: {visual_style}
    {style_instruction}
    Genre Context:
    {wiki_context[:CONFIG['context_length']]}
    {rag_context}
    Example Dialogue:
    {script_example}
    Output:"""

    def _call_generation_api(self, prompt: str) -> Dict:
        """Call the Gemini API to generate a storyboard with robust error handling"""
        # Set the headers for the HTTP request (specifying JSON content type)
        headers = {'Content-Type': 'application/json'}
        # Prepare the request payload with the prompt
        data = {"contents": [{"parts": [{"text": prompt}]}]}
        
        # Retry the API call up to the maximum number of retries
        for attempt in range(CONFIG["max_retries"]):
            try:
                # Construct the full API URL with the API key
                url = f"{CONFIG['api_url']}?key={api_key.value}"
                # Send a POST request to the Gemini API with a 30-second timeout
                response = requests.post(url, headers=headers, json=data, timeout=30)
                
                # Check for specific HTTP errors
                if response.status_code == 404:
                    raise ValueError("Invalid API endpoint")  # Endpoint not found
                if response.status_code == 403:
                    raise ValueError("API key rejected")  # API key issue
                
                # Raise an exception for other HTTP errors
                response.raise_for_status()
                # Parse the JSON response from the API
                result = response.json()
                
                # Validate the response format
                if 'candidates' not in result:
                    raise ValueError("Unexpected API response")
                
                # Extract the generated text from the response
                raw_output = result["candidates"][0]["content"]["parts"][0]["text"]
                # Use regex to extract the JSON portion of the response
                json_match = re.search(r'\{.*\}', raw_output, re.DOTALL)
                
                # If JSON is found, parse and validate it
                if json_match:
                    storyboard = json.loads(json_match.group(0))
                    if self.validate_storyboard(storyboard):
                        return storyboard  # Return the validated storyboard
                        
            except Exception as e:
                # On the last attempt, if it fails, return a fallback storyboard
                if attempt == CONFIG["max_retries"] - 1:
                    return self._create_fallback_storyboard(prompt.split('"')[1] if '"' in prompt else prompt)
        
        # If all retries fail, return a fallback storyboard
        return self._create_fallback_storyboard(prompt.split('"')[1] if '"' in prompt else prompt)

    def _create_fallback_storyboard(self, plot: str) -> Dict:
        """Create a minimal fallback storyboard when API generation fails"""
        # Return a basic storyboard structure with placeholder scenes
        return {
            "title": f"Untitled {plot}",  # Simple title based on the plot
            "scenes": [{
                "scene_number": i+1,  # Scene number (1 to 3)
                "description": f"Scene {i+1} of {plot}",  # Placeholder description
                "dialogue": "...",  # Placeholder dialogue
                "mood": "neutral"  # Default mood
            } for i in range(3)]  # Default to 3 scenes
        }

    def detect_genre(self, plot: str) -> str:
        """Detect the film genre from plot keywords using a keyword mapping"""
        # Convert the plot to lowercase for case-insensitive matching
        plot_lower = plot.lower()
        # Define a list of tuples mapping keywords to genres
        genre_map = [
            (["heist", "robbery", "steal"], "heist"),
            (["sci-fi", "futuristic", "space", "alien"], "sci-fi"),
            (["romance", "love", "relationship"], "romance"),
            (["thriller", "suspense", "mystery"], "thriller")
        ]
        
        # Check for keyword matches in the plot
        for keywords, genre in genre_map:
            if any(word in plot_lower for word in keywords):
                return genre  # Return the matched genre
        return CONFIG["default_genre"]  # Default to 'drama' if no match

    def fetch_wikipedia_film_data(self, genre: str) -> str:
        """Fetch film context from Wikipedia with error handling for genre-specific data"""
        # Construct the Wikipedia URL for the genre
        url = f"https://en.wikipedia.org/wiki/{genre}_film"
        try:
            # Send a GET request to Wikipedia with a 10-second timeout
            response = requests.get(url, timeout=10)
            # Raise an exception for HTTP errors
            response.raise_for_status()
            
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            # Find the main content div
            content = soup.find('div', {'id': 'mw-content-text'})
            # Extract the first 3 paragraphs
            paragraphs = content.find_all('p')[:3] if content else []
            
            # Clean and join the paragraph text, limiting to context_length
            return " ".join([p.get_text().strip() for p in paragraphs if p.get_text().strip()])[:CONFIG["context_length"]]
        except Exception as e:
            # Display a warning if Wikipedia fetch fails
            display(Markdown(f"⚠️ **Wikipedia Unavailable:** Using generic context. (Error: {str(e)})"))
            return f"Standard {genre} film context."  # Fallback context

    def fetch_script_data(self, genre: str) -> str:
        """Fetch script snippets from a mock database based on the genre"""
        # Define a mock database of script snippets for different genres
        mock_scripts = {
            "heist": "INT. BANK VAULT - NIGHT\nThe crew works silently until alarms blare.\n\"We're made!\" shouts the leader.",
            "sci-fi": "EXT. SPACE STATION\nThe captain watches Earth from the viewport.\n\"Initiate hyperdrive,\" she orders.",
            "romance": "EXT. PARIS CAFE - DAY\nThey share coffee as rain falls gently.\n\"I've waited my whole life for this,\" he whispers.",
            "thriller": "INT. ABANDONED HOUSE - NIGHT\nA floorboard creaks. She holds her breath.\n\"I know you're here,\" calls the killer."
        }
        # Return the script for the genre, or a default if not found
        return mock_scripts.get(genre.lower(), "Sample script dialogue.")

    def analyze_mood(self, storyboard: Dict) -> Dict:
        """Analyze the mood distribution in the storyboard for visualization"""
        # Extract moods from each scene in the storyboard
        moods = [scene.get("mood", "unknown") for scene in storyboard.get("scenes", [])]
        # Count the frequency of each mood
        return {mood: moods.count(mood) for mood in set(moods)} if moods else {}

    def validate_storyboard(self, storyboard: Dict) -> bool:
        """Validate the structure of the generated storyboard to ensure it meets requirements"""
        # Check if the storyboard has a title and scenes
        if not isinstance(storyboard, dict) or "title" not in storyboard or "scenes" not in storyboard:
            return False
        # Check if scenes is a non-empty list
        if not isinstance(storyboard["scenes"], list) or not storyboard["scenes"]:
            return False
        # Validate each scene's structure
        for scene in storyboard["scenes"]:
            if not all(key in scene for key in ["scene_number", "description", "dialogue", "mood"]):
                return False
            if scene["mood"] not in CONFIG["moods"]:
                return False
        return True  # Return True if all checks pass

    def save_storyboard(self, storyboard: Dict, filename: str) -> bool:
        """Save the storyboard to a JSON file and return success status"""
        try:
            # Write the storyboard to the specified file with proper formatting
            with open(filename, 'w') as f:
                json.dump(storyboard, f, indent=2)
            return True  # Return True if saving succeeds
        except Exception as e:
            # Display an error if saving fails
            display(Markdown(f"⚠️ **Error saving storyboard:** {str(e)}"))
            return False  # Return False if saving fails

## 🧠 RAG Implementation
The Retrieval-Augmented Generation system provides contextual enhancement:

In [ ]:
# Extend the StoryboardAgent class with RAG methods
def retrieve_similar_plots(self, plot: str, top_k: int = 3) -> List[Dict]:
    """Retrieve similar plots from the knowledge base using vector similarity"""
    # Check if the embedding model is available
    if not self.embedding_model:
        return []  # Return empty list if embedding model failed to load
    
    # Load the knowledge base from the JSON file
    with open(CONFIG["knowledge_base"], "r") as f:
        kb = json.load(f)
    
    # Check if there are any plots in the knowledge base
    if not kb["plots"]:
        return []  # Return empty list if knowledge base is empty
    
    # Generate an embedding for the input plot
    plot_embedding = self.embedding_model.encode(plot)
    
    # Calculate cosine similarities between the input plot and stored plots
    similarities = []
    for stored_plot in kb["plots"]:
        stored_embedding = np.array(stored_plot["embedding"])
        # Compute cosine similarity between the input and stored embeddings
        similarity = cosine_similarity(
            [plot_embedding],
            [stored_embedding]
        )[0][0]
        similarities.append((stored_plot, similarity))
    
    # Sort by similarity and filter by threshold, returning the top_k matches
    return [
        plot for plot, sim in sorted(similarities, key=lambda x: x[1], reverse=True)
        if sim > CONFIG["rag_threshold"]
    ][:top_k]

def _update_knowledge_base(self, plot: str, storyboard: Dict):
    """Update the knowledge base with a new plot and its storyboard"""
    try:
        # Load the existing knowledge base
        with open(CONFIG["knowledge_base"], "r") as f:
            kb = json.load(f)
        
        # Generate an embedding for the plot if the embedding model is available
        if self.embedding_model:
            plot_embedding = self.embedding_model.encode(plot).tolist()
        else:
            plot_embedding = []  # Use empty list if embedding model is unavailable
        
        # Add the new plot and storyboard to the knowledge base
        kb["plots"].append({
            "plot": plot,  # Store the plot text
            "storyboard": storyboard,  # Store the generated storyboard
            "embedding": plot_embedding,  # Store the plot embedding
            "timestamp": str(datetime.now())  # Add a timestamp for the entry
        })
        
        # Save the updated knowledge base back to the file
        with open(CONFIG["knowledge_base"], "w") as f:
            json.dump(kb, f, indent=2)
            
    except Exception as e:
        # Display an error if updating the knowledge base fails
        display(Markdown(f"⚠️ Could not update knowledge base: {e}"))

# Attach the methods to the StoryboardAgent class
StoryboardAgent.retrieve_similar_plots = retrieve_similar_plots
StoryboardAgent._update_knowledge_base = _update_knowledge_base

## 📊 Visualization Enhancements
Professional visualizations with improved styling:

In [ ]:
def visualize_mood(mood_counts: Dict):
    """Create an enhanced mood visualization, save it as an image, and display it"""
    # Debug: Print the mood counts to ensure data is correct
    print(f"Debug: Mood counts received - {mood_counts}")
    
    # Check if there is mood data to visualize
    if not mood_counts:
        display(Markdown("⚠️ No mood data available"))
        return

    # Prepare data for visualization
    moods = list(mood_counts.keys())  # List of moods
    counts = list(mood_counts.values())  # List of counts for each mood

    # Create a new figure for the plot
    plt.figure(figsize=(10, 6))  # Set the figure size
    ax = plt.gca()  # Get the current axis

    # Define a color mapping for moods to make the chart visually appealing
    color_map = {
        "joyful": "#FFD700", "romantic": "#FF69B4", "tense": "#8B0000",
        "suspenseful": "#4B0082", "dark": "#000000", "hopeful": "#32CD32",
        "chaotic": "#FF4500"
    }

    # Assign colors to each mood, defaulting to gray if not in the map
    colors = [color_map.get(mood, "#888888") for mood in moods]

    # Create a bar chart with the mood counts
    bars = ax.bar(moods, counts, color=colors, edgecolor='white', linewidth=1)

    # Style the chart
    ax.set_title("Scene Mood Distribution", fontsize=16, pad=20, fontweight='bold')  # Set the title
    ax.set_xlabel("Mood Type", fontsize=12)  # Set the x-axis label
    ax.set_ylabel("Number of Scenes", fontsize=12)  # Set the y-axis label
    ax.grid(axis='y', linestyle='--', alpha=0.3)  # Add a light grid on the y-axis
    ax.spines[['top', 'right']].set_visible(False)  # Remove top and right spines for cleaner look

    # Add value labels on top of each bar
    for bar in bars:
        height = bar.get_height()  # Get the height of the bar
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',  # Display the count
                ha='center', va='bottom',  # Center horizontally, place above the bar
                fontsize=11, fontweight='bold')  # Set font size and weight

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.tight_layout()  # Adjust layout to prevent clipping

    # Save the plot as an image file
    chart_filename = "mood_distribution.png"
    plt.savefig(chart_filename, bbox_inches='tight')
    plt.close()  # Close the plot to prevent duplicate display

    # Display the saved image using Markdown
    display(Markdown(f"![Mood Distribution Chart]({chart_filename})"))

def display_storyboard(storyboard: Dict):
    """Render the storyboard in plain Markdown format without HTML styling"""
    # Display the storyboard title
    display(Markdown(f"## 🎬 {storyboard.get('title', 'Untitled Storyboard')}"))
    
    # Display each scene in a simple Markdown format
    for scene in storyboard.get("scenes", []):
        mood = scene.get("mood", "neutral").lower()
        display(Markdown(f"""
### 🎥 Scene {scene.get('scene_number', 1)} ({mood.capitalize()})

**Visual Description:**  
{scene.get('description', 'No description available')}

**Dialogue:**  
"{scene.get('dialogue', '...')}"
"""))

# 🖥️ Enhanced User Interface
The UI now includes progress tracking and better visual feedback:

In [ ]:
# Create enhanced input widgets for user interaction
plot_input = widgets.Textarea(
    placeholder='Describe your story (e.g., "A detective discovers aliens in 1920s Chicago")',  # Placeholder text for plot input
    description='Plot:',  # Label for the plot input field
    layout=Layout(width='80%', height='100px'),  # Set the size of the textarea
    style={'description_width': 'initial'}  # Adjust the label width for better alignment
)

scenes_slider = widgets.IntSlider(
    value=3,  # Default number of scenes
    min=1,  # Minimum number of scenes
    max=CONFIG["max_scenes"],  # Maximum number of scenes (from CONFIG)
    step=1,  # Increment step for the slider
    description='Number of Scenes:',  # Label for the slider
    continuous_update=False,  # Only update on release for better performance
    style={'description_width': 'initial'}  # Adjust label width
)

style_selector = widgets.Dropdown(
    options=['Cinematic', 'Documentary', 'Anime', 'Noir', 'Experimental'],  # Options for visual style
    value='Cinematic',  # Default style
    description='Visual Style:',  # Label for the dropdown
    style={'description_width': 'initial'}  # Adjust label width
)

generate_btn = widgets.Button(
    description='Generate Storyboard',  # Button text
    button_style='success',  # Green button style
    layout=Layout(width='200px', height='40px'),  # Set button size
    icon='film'  # Add a film icon to the button
)

output_area = widgets.Output()  # Create an output area for displaying results

def on_generate_click(b):
    """Enhanced click handler with progress tracking for the generate button"""
    with output_area:
        # Clear the output area to start fresh
        output_area.clear_output()

        # Create a progress bar to show generation steps
        steps = widgets.HBox([
            widgets.Label(value="Progress:"),  # Label for the progress bar
            widgets.IntProgress(
                value=0,  # Initial progress value
                min=0,  # Minimum progress value
                max=4,  # Maximum progress steps (4 steps)
                description='',  # No description (label is enough)
                bar_style='info'  # Blue progress bar style
            )
        ])
        # Display the progress bar
        display(steps)
        
        # Initialize the StoryboardAgent to handle generation tasks
        agent = StoryboardAgent()
        
        try:
            # Step 1: Preparation - Update progress and show message
            steps.children[1].value = 1
            display(Markdown("### 🔍 Analyzing your plot..."))
            
            # Step 2: Generation - Generate the storyboard using the agent
            steps.children[1].value = 2
            display(Markdown("### 🎥 Generating storyboard..."))
            storyboard = agent.execute_function(
                "generate_storyboard",
                plot=plot_input.value,  # Use the user-provided plot
                num_scenes=scenes_slider.value  # Use the selected number of scenes
            )
            
            # Check if the storyboard was generated successfully
            if not storyboard:
                raise ValueError("Generation failed")
            
            # Step 3: Analysis - Analyze the mood distribution
            steps.children[1].value = 3
            display(Markdown("### 📊 Analyzing story structure..."))
            mood_analysis = agent.execute_function(
                "analyze_mood",
                storyboard=storyboard  # Pass the generated storyboard
            )
            
            # Step 4: Display - Show the results
            steps.children[1].value = 4
            display(Markdown(f"## 🎬 {storyboard.get('title', 'Your Storyboard')}"))
            display_storyboard(storyboard)  # Display the formatted storyboard
            visualize_mood(mood_analysis)  # Visualize the mood distribution
            
            # Save the storyboard to a file
            filename = f"storyboard_{plot_input.value[:20].replace(' ', '_')}.json"
            if agent.execute_function("save_storyboard", storyboard=storyboard, filename=filename):
                # Display a download button for the saved JSON file
                display(Markdown("### 💾 Download Options"))
                display(HTML(f"""
                <a href="{filename}" download>
                    <button style="background-color:#4CAF50; color:white; padding:10px 20px; border:none; border-radius:6px; font-size:14px; margin:5px">
                        Download JSON
                    </button>
                </a>
                """))
            
            # Mark the progress as complete
            steps.children[1].bar_style = 'success'
            
        except Exception as e:
            # If an error occurs, mark the progress as failed and show the error
            steps.children[1].bar_style = 'danger'
            display(Markdown(f"## ❌ Error: {str(e)}"))
            display(Markdown("Please try again with a different plot description."))

# Connect the click handler to the generate button
generate_btn.on_click(on_generate_click)

# Display the enhanced UI components
display(Markdown("### ✍️ Your Story Parameters"))
display(widgets.VBox([
    plot_input,  # Plot input textarea
    widgets.HBox([scenes_slider, style_selector]),  # Horizontal layout for slider and dropdown
    generate_btn  # Generate button
]))
display(output_area)  # Output area for results

## 📖 How to Use

<div style="text-align: center;">
    <table style="margin: 0 auto; border-collapse: collapse; width: 80%;">
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid #ddd; padding: 8px;">Step</th>
            <th style="border: 1px solid #ddd; padding: 8px;">Action</th>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">1</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Enter API Key - Get from Google AI Studio</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">2</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Describe Your Story - Be specific about setting and characters</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">3</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Adjust Parameters - Select scene count and visual style</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">4</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Generate - Create your storyboard</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">5</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Download - Save as JSON for further editing</td>
        </tr>
    </table>
</div>

## 🛠️ Troubleshooting

<div style="text-align: center;">
    <table style="margin: 0 auto; border-collapse: collapse; width: 80%;">
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid #ddd; padding: 8px;">Issue</th>
            <th style="border: 1px solid #ddd; padding: 8px;">Solution</th>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">API Errors</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Check key validity and quota</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">Empty Output</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Simplify plot description</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">Format Issues</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Restart kernel and retry</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 8px;">Slow Performance</td>
            <td style="border: 1px solid #ddd; padding: 8px;">Reduce scene count</td>
        </tr>
    </table>
</div>

## 🚀 Future Enhancements

- Image generation integration
- Collaborative editing
- Script formatting options
- Character development tools